# Streak Detection and Correction

## Background

Occasionally, due to malfunctions caused by the instrument there are traces of horizontal steaks throughout each channel across an image. Each are one pixel in height and vary in length. These streaks are present in every channel with varying degrees of visibility. The purpose of this notebook is to utilize a suite of to remove these streaks from MIBI images.

Here is what you will need to make full use of this notebook.

1. A set of fovs, or just one fov with these streaks. In your tiff viewer of choice, you may need to adjust the exposure and gamma to make sure that you can see them with your own eyes. (do not save the exposure and gamma changes to the tiff image).
2. This notebook. Potentially this will become another preprocessing step in the Rosetta pipeline however it can be ran here as well. 

In [1]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
from ark.utils import load_utils

import sys
sys.path.append("../")
from toffy import streak_detection as sd

## 1. Load the FOV(s) paths

Add the path to the directory containing the fovs.

In [ ]:
# The location of the fovs
path_to_fovs: str = "your/path/to/a/folder/of/fovs/"
data_dir = Path(path_to_fovs)
fovs = [fov.stem for fov in data_dir.glob("*//") if "corrected" not in fov.stem]

## 2. Run Streak Detection + Correction

1. Choose the channel you would like to use for streak detection for `streak_channel`. It defaults to **Noodle**. 
2. Choose if you would like to save the mask metadata for visualization purposes. In your tiff viewer of choice you would take the masks you wish to look at and lay them on top of the corrected image or the original image. This can be changed with the `visualization_masks` variable. It defaults to **False**.

In [ ]:
streak_channel = "Noodle"
visualization_masks = False

In [ ]:
for fov in tqdm(fovs):

    # Load fov
    fov_data = load_utils.load_imgs_from_tree(data_dir=data_dir, fovs=[fov], dtype=np.int32)

    # Correct streaks
    corrected_channels, streak_data = sd.streak_correction(
        fov_data=fov_data, streak_channel=streak_channel, visualization_masks=visualization_masks
    )

    # Save the corrected channels per fov.
    sd.save_corrected_channels(
        streak_data=streak_data,
        corrected_channels=corrected_channels,
        data_dir=data_dir,
        save_streak_data=True,
    )
